In [130]:
import pandas as pd
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
import category_encoders as ce

from numpy import mean
from numpy import std

In [ ]:
'''
using the perceptron alg to predict hotel cancellations
'''

In [113]:
hotels_df = pd.read_csv('hotel.csv')
hotels_df.shape

(119390, 32)

In [114]:
hotels_df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [5]:
hotels_df['hotel'].value_counts()
for col in hotels_df.columns:
    print(col)

hotel
is_canceled
lead_time
arrival_date_year
arrival_date_month
arrival_date_week_number
arrival_date_day_of_month
stays_in_weekend_nights
stays_in_week_nights
adults
children
babies
meal
country
market_segment
distribution_channel
is_repeated_guest
previous_cancellations
previous_bookings_not_canceled
reserved_room_type
assigned_room_type
booking_changes
deposit_type
agent
company
days_in_waiting_list
customer_type
adr
required_car_parking_spaces
total_of_special_requests
reservation_status
reservation_status_date


In [115]:
hotels_df = hotels_df.drop(['meal','distribution_channel','market_segment','company','assigned_room_type','reserved_room_type', 'reservation_status_date'], axis = 1)

In [116]:
hotels_df = hotels_df.dropna()
hotels_df.shape

(102894, 25)

In [117]:
X = hotels_df.copy()
y = X.pop('is_canceled')


#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['arrival_date_month'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X) 
#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['hotel'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X)
#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['deposit_type'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X)
#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['customer_type'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X)
#Create object for binary encoding
encoder= ce.BinaryEncoder(cols=['reservation_status'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X)
encoder= ce.BinaryEncoder(cols=['country'],return_df=True)
#Fit and Transform Data 
X = encoder.fit_transform(X)

X.head()

,hotel_0,hotel_1,lead_time,arrival_date_year,arrival_date_month_0,arrival_date_month_1,arrival_date_month_2,arrival_date_month_3,arrival_date_week_number,arrival_date_day_of_month,...,agent,days_in_waiting_list,customer_type_0,customer_type_1,customer_type_2,adr,required_car_parking_spaces,total_of_special_requests,reservation_status_0,reservation_status_1
3,0,1,13,2015,0,0,0,1,27,1,...,304.0,0,0,0,1,75.0,0,0,0,1
4,0,1,14,2015,0,0,0,1,27,1,...,240.0,0,0,0,1,98.0,0,1,0,1
5,0,1,14,2015,0,0,0,1,27,1,...,240.0,0,0,0,1,98.0,0,1,0,1
7,0,1,9,2015,0,0,0,1,27,1,...,303.0,0,0,0,1,103.0,0,1,0,1
8,0,1,85,2015,0,0,0,1,27,1,...,240.0,0,0,0,1,82.0,0,1,1,0


In [41]:
'''
~this is code for explicitly mapping the vals~
X['arrival_date_month'] = \
    X['arrival_date_month'].map(
        {'January':1, 'February': 2, 'March':3,
         'April':4, 'May':5, 'June':6, 'July':7,
         'August':8, 'September':9, 'October':10,
         'November':11, 'December':12}
    )

X['hotel'] = \
    X['hotel'].map(
          {'City Hotel':1, 'Resort Hotel':2}
)
X['deposit_type'] = \
    X['deposit_type'].map(
          {'No Deposit':1, 'Non Refund':2, 'Refundable':3}
)
X['customer_type'] = \
    X['customer_type'].map(
          {'Transient':1, 'Transient-Party':2, 'Contract':3, 'Group':4}
)
X['reservation_status'] = \
    X['reservation_status'].map(
          {'Check-Out':1, 'Canceled':2, 'No-Show':3}
)
'''

"\n#Create object for binary encoding\nencoder= ce.BinaryEncoder(cols=['hotel'],return_df=True)\n#Fit and Transform Data \nX = encoder.fit_transform(X)\n"

In [118]:
print(X.shape, y.shape)

(102894, 39) (102894,)


In [119]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [120]:
sc = StandardScaler()
sc.fit(X_train)

StandardScaler()

In [121]:
# Apply the scaler to the X training data
X_train_std = sc.transform(X_train)

# Apply the SAME scaler to the X test data
X_test_std = sc.transform(X_test)

In [143]:
# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
ppn = Perceptron(max_iter = 50, eta0=0.1, random_state=0) #need to figure out why my accuracy is....

# Train the perceptron
ppn.fit(X_train, y_train)

Perceptron(eta0=1, max_iter=50)

In [144]:
# Apply the trained perceptron on the X data to make predicts for the y test data
y_pred = ppn.predict(X_test_std)

C:\Users\lucie\Miniconda3\envs\mlvenv\lib\site-packages\sklearn\base.py:441: UserWarning: X does not have valid feature names, but Perceptron was fitted with feature names
  warnings.warn(


In [145]:
y_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [146]:
y_test

110563    0
81449     0
7416      1
116523    0
111808    0
         ..
96272     0
73712     1
49593     1
70109     1
80884     0
Name: is_canceled, Length: 30869, dtype: int64

In [147]:
# View the accuracy of the model, which is: 1 - (observations predicted wrong / total observations)
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))

Accuracy: 1.00


In [148]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(ppn, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# summarize result
print('Mean Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Mean Accuracy: 0.846 (0.193)


In [ ]:
'''
sources: https://chrisalbon.com/code/machine_learning/basics/perceptron_in_scikit-learn/
https://www.kaggle.com/lucienmensah/exercise-binary-classification/edit
https://machinelearningmastery.com/perceptron-algorithm-for-classification-in-python/
'''